<a href="https://colab.research.google.com/github/Kekoa-Pickett/ML-HW2/blob/main/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 4

For this assignment, you will be developing an artificial neural network to classify data given in the __[Dry Beans Data Set](https://archive.ics.uci.edu/ml/datasets/Dry+Bean+Dataset#)__. This data set was obtained as a part of a research study by Selcuk University, Turkey, in which a computer vision system was developed to distinguish seven different registered varieties of dry beans with similar features. More details on the study can be found in the following __[research paper](https://www.sciencedirect.com/science/article/pii/S0168169919311573)__. <br>
### **Make sure to use the lecture notebook on an introduction to keras and cross validation located [here](https://colab.research.google.com/drive/1ksEGL7SJ_wutCIyPYx7Loe5EPdOij6dJ?usp=sharing) and [here](https://colab.research.google.com/drive/1C9Mwf1J2ril1Q4l6n2BjQMb8YaFySG5_?usp=sharing)**.

## About the Data Set
Seven different types of dry beans were used in a study in Selcuk University, Turkey, taking into account the features such as form, shape, type, and structure by the market situation. A computer vision system was developed to distinguish seven different registered varieties of dry beans with similar features in order to obtain uniform seed classification. For the **classification** model, images of 13611 grains of 7 different registered dry beans were taken with a high-resolution camera. Bean images obtained by computer vision system were subjected to segmentation and feature extraction stages, and a total of 16 features - 12 dimensions and 4 shape forms - were obtained from the grains.

Number of Instances (records in the data set): __13611__

Number of Attributes (fields within each record, including the class): __17__

### Data Set Attribute Information:

1. __Area (A)__ : The area of a bean zone and the number of pixels within its boundaries.
2. __Perimeter (P)__ : Bean circumference is defined as the length of its border.
3. __Major axis length (L)__ : The distance between the ends of the longest line that can be drawn from a bean.
4. __Minor axis length (l)__ : The longest line that can be drawn from the bean while standing perpendicular to the main axis.
5. __Aspect ratio (K)__ : Defines the relationship between L and l.
6. __Eccentricity (Ec)__ : Eccentricity of the ellipse having the same moments as the region.
7. __Convex area (C)__ : Number of pixels in the smallest convex polygon that can contain the area of a bean seed.
8. __Equivalent diameter (Ed)__ : The diameter of a circle having the same area as a bean seed area.
9. __Extent (Ex)__ : The ratio of the pixels in the bounding box to the bean area.
10. __Solidity (S)__ : Also known as convexity. The ratio of the pixels in the convex shell to those found in beans.
11. __Roundness (R)__ : Calculated with the following formula: (4piA)/(P^2)
12. __Compactness (CO)__ : Measures the roundness of an object: Ed/L
13. __ShapeFactor1 (SF1)__
14. __ShapeFactor2 (SF2)__
15. __ShapeFactor3 (SF3)__
16. __ShapeFactor4 (SF4)__

17. __Classes : *Seker, Barbunya, Bombay, Cali, Dermosan, Horoz, Sira*__

### Libraries that can be used :
- NumPy, SciPy, Pandas, Sci-Kit Learn, TensorFlow, Keras. You may also use PyTorch (though support may be limited)
- Any other library used during the lectures and discussion sessions.

### Other Notes
- Don't worry about not being able to achieve high accuracy, it is neither the goal nor the grading standard of this assignment.
- Discussion and Lecture materials should be helpful for doing the assignments.
- The homework submission should be a .ipynb file.


In [1]:
!git clone https://github.com/ucsd-cse151a-f24/hw4.git

Cloning into 'hw4'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (60/60), done.
Receiving objects: 100% (63/63), 1.58 MiB | 1.19 MiB/s, done.
remote: Total 63 (delta 36), reused 7 (delta 2), pack-reused 0 (from 0)
Resolving deltas: 100% (36/36), done.



## Exercise 1 : Building a Feed-Forward Neural Network(50 points)

### Exercise 1.1 : Data Preprocessing (10 points)

- As the classes are categorical, use one-hot encoding to represent the set of classes. You will find this useful when developing the output layer of the neural network.
- Split the data into training and testing set by __90:10__ and use the training set for training the model and the test set to evaluate the model performance. Please set verbose=0 to suppress output during training.
- Normalize each field of the input data using the min-max normalization technique.

__Notes:__

- Splitting of the dataset should be done __before__ the normalization step and __after__ the one-hot encoding.

In [2]:
!pip install scikeras
!pip install keras-tuner -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 5.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

df = pd.read_csv('hw4/Dry_Beans_Dataset.csv')

oneHot = OneHotEncoder(sparse_output=False)
oneHot_df = oneHot.fit_transform(df[['Class']])
oneHot_feature_names = oneHot.get_feature_names_out(['Class'])

enc_df = pd.DataFrame(oneHot_df, columns=oneHot_feature_names)
df = pd.concat([df, enc_df], axis=1)
df = df.drop('Class', axis=1)

train_x, test_x, train_y, test_y = train_test_split(df.drop(oneHot_feature_names, axis=1), df[oneHot_feature_names], test_size=0.1, random_state=42)

minMax = MinMaxScaler()
train_x = minMax.fit_transform(train_x)
test_x = minMax.transform(test_x)
train_y = minMax.fit_transform(train_y)
test_y = minMax.transform(test_y)

print(train_x.shape)
print(train_y)
print(test_x.shape)
print(test_y)



(12249, 16)
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]
(1362, 16)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]]


### Exercise 1.2 : Training and Testing the Neural Network (40 points)

Design a 3-layer (3 hidden layers and this does not include the input or output layer) artificial deep neural network, specifically a feed-forward multi-layer perceptron (using the sigmoid activation function), to classify the type of 'Dry Bean' given the other attributes in the data set, similar to the one mentioned in the paper above. Please note that this is a **multi-class classification** problem so select the right number of nodes accordingly for the input and output layers.

Consider the following hyperparameters while developing your model:

- Model type: Keras Sequential
- Make sure your input layer matches the size of your X matrix
- Number and type of hidden layers: 3 and Dense
- Number of nodes in each hidden layer: 12
- Learning rate should be 0.3
- Number of epochs should be 100
- The sigmoid function is to be used as the activation function in each layer
- Your output layer has to use a sigmoid function and the number of outputs should match the shape of your y
- Your loss function should be MSE
- Stochastic Gradient Descent should be used to minimize the error rate

**Note:** We are having you use MSE as your loss function for this model, is this a good choice? Why or why not? If not, what should you use instead in future models? Answer below

**ANSWER:** This is not a good idea because there are multiple different output classes so the error will most likely be super high because it is being compared to so many different things. We could use log loss since this is a classification problem.

__Requirements once the model has been trained :__

- A confusion matrix for all classes, specifying the true positive, true negative, false positive, and false negative cases for each category in the class
- Since we do have OHE output (multi-class output) you will need to either reshape or argmax your outputs. Make sure they have already been thresholded as well i.e. look at yhat and do you see 1's and 0's?
- The accuracy and mean squared error (MSE) of the model
- The precision and recall for each label in the class

__Notes :__

- The mean squared error (MSE) values obtained __should be positive__.


In [4]:
import tensorflow as tf
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error

def buildmodel():
    model = Sequential([
        Dense(12, activation = 'sigmoid', input_dim = train_x.shape[1]), # Here our input dimension is set inside the first hidden layer with input_dim = X.shape[1]
        Dense(12, activation = 'sigmoid'),
        Dense(12, activation = 'sigmoid'),
        Dense(12, activation = 'sigmoid'),
        Dense(7, activation = 'sigmoid')
    ])
    optimizer = SGD(learning_rate=0.3)
    model.compile(optimizer = optimizer, loss='mse', metrics=['mse'])
    return(model)


neural_network = buildmodel
estimator = KerasClassifier(model=neural_network, epochs=20, verbose=0)
estimator.fit(train_x, train_y, verbose=0)
yhat_probs = estimator.predict(test_x)
threshold = np.mean(yhat_probs)
yhat = (yhat_probs >= threshold).astype(int)
yhat_labels = np.argmax(yhat, axis=1)
test_y_labels = np.argmax(test_y, axis=1)

confusion_matrix_result = confusion_matrix(test_y_labels, yhat_labels)
print(confusion_matrix_result)
print(classification_report(test_y_labels, yhat_labels))
print(mean_squared_error(test_y_labels, yhat_labels))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[[  0   0   0 137   0   0   0]
 [  0   0   0  63   0   0   0]
 [  0   0   0 195   0   0   0]
 [  0   0   0 342   0   0   0]
 [  0   0   0 181   0   0   0]
 [  0   0   0 200   0   0   0]
 [  0   0   0 244   0   0   0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       137
           1       0.00      0.00      0.00        63
           2       0.00      0.00      0.00       195
           3       0.25      1.00      0.40       342
           4       0.00      0.00      0.00       181
           5       0.00      0.00      0.00       200
           6       0.00      0.00      0.00       244

    accuracy                           0.25      1362
   macro avg       0.04      0.14      0.06      1362
weighted avg       0.06      0.25      0.10      1362

3.566079295154185


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Exercise 2 : k-fold Cross Validation (20 points)

In order to avoid **using biased models**, use 10-fold cross validation to generalize the model from Ex1.2 on the given data set. You can choose a n_repeats value of 1-5

__Requirements :__
- Print the accuracy values during each iteration of the **cross validation** not the iterations per epoch or the epochs
- Print the overall average accuracy per each n_fold value, look at the documentation for the scoring parameter



In [5]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score

# epochs are the number of iterative training steps we do on the entire dataset, and each iteration is a step. batch_size is the number of datapoints we select per batch. total obs = batch_size * batches
kfold = RepeatedKFold(n_splits = 10, n_repeats = 2, random_state= 21)
results = cross_val_score(estimator, train_x, train_y, cv=kfold, n_jobs = 1)  # 1 cpus
results.mean()  # Mean MSE for all splits and repeated runs.
scores = cross_val_score(estimator, train_x, train_y, scoring='accuracy', cv=kfold, n_jobs=-1)

print(scores)
print(np.mean(scores))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/lo

[0.26367347 0.26040816 0.26367347 0.25877551 0.26122449 0.25061224
 0.25061224 0.25632653 0.28       0.27042484 0.25959184 0.28734694
 0.25061224 0.26040816 0.2555102  0.26693878 0.23265306 0.26204082
 0.24       0.30065359 0.27510204 0.26285714 0.27428571 0.25469388
 0.27346939 0.25795918 0.25469388 0.25469388 0.26122449 0.24673203
 0.24       0.25714286 0.28653061 0.26367347 0.27591837 0.27183673
 0.27102041 0.24897959 0.25714286 0.24346405 0.27346939 0.25142857
 0.25306122 0.26040816 0.27510204 0.25061224 0.27428571 0.26040816
 0.2522449  0.26470588]
0.26157266906762705


## Exercise 3 : Hyperparameter Tuning (25 points)

Use either grid search or random search methodology to find the optimal number of nodes required in each hidden layer, as well as the optimal learning rate and the different activation functions or optimization approaches, [keras_tuner examples](https://keras.io/guides/keras_tuner/getting_started/) such that the accuracy of the model is maximum for the given data set.

__Requirements :__
- The set of optimal hyperparameters
- Try your best to maximize accuracy using this set of optimal hyperparameters

__Note :__ Hyperparameter tuning takes a lot of time to execute. Make sure that you choose the appropriate number of each hyperparameter (preferably 3 of each), and that you allocate enough time to execute your code. Make sure to tune at least three parameters with three options each at a minimum, but feel free to experiment with more, just recognize that it will grow exponentially in running time

In [12]:
import keras_tuner
from keras_tuner.tuners import RandomSearch
#from keras_tuner.engine.hyperparameters import HyperParameters #No need to import it seprately
import keras
from keras import layers

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    for i in range(hp.Int("num_layers", 1, 4)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=hp.Choice("activation", ["relu", "tanh", "sigmoid"]),
            )
        )
    model.add(layers.Dense(7, activation="softmax"))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=4,
    executions_per_trial=2,
    overwrite=True,
)

tuner.search(train_x, train_y, epochs=2, validation_data=(test_x, test_y))
models = tuner.get_best_models(num_models=2)
best_model = models[0]
print(best_model.summary())
print(tuner.results_summary())

Trial 3 Complete [00h 00m 19s]
val_accuracy: 0.8906020522117615

Best val_accuracy So Far: 0.8931717872619629
Total elapsed time: 00h 00m 39s


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 384)                 │           6,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │          12,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 7)                   │             231 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,135 (78.65 KB)

 Trainable params: 20,135 (78.65 KB)

 Non-trainable params: 0 (0.00 B)

## Exercise 4 - Collaborative Statement (5 points)

It is mandatory to include a Statement of Collaboration in each submission, that follows the guidelines below.
Include the names of everyone involved in the discussions (especially in-person ones), and what was discussed.
All students are required to follow the academic honesty guidelines posted on the course website. For
programming assignments in particular, I encourage students to organize (perhaps using Piazza) to discuss the
task descriptions, requirements, possible bugs in the support code, and the relevant technical content before they
start working on it. However, you should not discuss the specific solutions, and as a guiding principle, you are
not allowed to take anything written or drawn away from these discussions (no photographs of the blackboard,
written notes, referring to Piazza, etc.). Especially after you have started working on the assignment, try to restrict
the discussion to Piazza as much as possible, so that there is no doubt as to the extent of your collaboration.

Even if you did not use any outside resources or collaborate with anyone, please state that explicitly in the space below.

I didn't talk about or look at anyones answers for this assignment.

In [7]:
# yhat = neural_network.predict(test_x)
# yhat = np.argmax(yhat, axis=1)
# test_y = np.argmax(test_y, axis=1)
# yhat = np.argmax(yhat, axis=1)
# test_y = np.argmax(test_y, axis=1)
# import threshold
# yhat

AttributeError: 'function' object has no attribute 'predict'